In [1]:
pip install sqlalchemy ipython-sql

Note: you may need to restart the kernel to use updated packages.


### Connect to the database

In [2]:
!pip install ipython-sql
!pip install ipython-sql prettytable

In [3]:
%load_ext sql

In [4]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
!pip install -q pandas

In [6]:
%sql sqlite:///my_data1.db

In [7]:
import pandas as pd 
csv_path = "/Users/estellepan/Desktop/Spacex.csv"
df = pd.read_csv(csv_path)
# Save into SQLite database
df.to_sql("SPACEXTBL", con, if_exists="replace", index=False, method="multi")

101

In [8]:
#DROP THE TABLE IF EXISTS
%sql DROP TABLE IF EXISTS SPACEXTABLE;
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.
 * sqlite:///my_data1.db
Done.


[]

##### Display the names of the unique launch sites  in the space mission

In [9]:
%%sql
SELECT DISTINCT Launch_Site
FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


##### Display 5 records where launch sites begin with the string 'CCA'

In [10]:
%%sql
SELECT *
FROM SPACEXTABLE
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


##### Total payload mass carried by boosters launched by NASA (CRS)

In [11]:
%%sql
SELECT SUM("Payload_Mass__kg_") AS Total_Payload
FROM SPACEXTABLE
WHERE Customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


Total_Payload
45596


##### Average payload mass carried by booster version F9 v1.1

In [12]:
%%sql
SELECT AVG("Payload_Mass__kg_") AS Avg_Payload
FROM SPACEXTABLE
WHERE Booster_Version = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


Avg_Payload
2928.4


##### The date when the first succesful landing outcome in ground pad was acheived.

In [13]:
%%sql
SELECT MIN(Date) AS First_Successful_Landing
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


First_Successful_Landing
2015-12-22


##### Names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [14]:
%%sql
SELECT DISTINCT Booster_Version
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (drone ship)'
  AND "Payload_Mass__kg_" BETWEEN 4000 AND 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


##### Total number of successful and failure mission outcomes

In [15]:
%%sql
SELECT Mission_Outcome, COUNT(*) AS total
FROM SPACEXTABLE
GROUP BY Mission_Outcome;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,total
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


##### Yearly Launch Counts and Success Rates (2010–2020)

In [16]:
%%sql
SELECT substr(Date, 1, 4) AS Year,
       COUNT(*) AS Total_Launches,
       SUM(CASE WHEN "Landing_Outcome" LIKE 'Success%' THEN 1 ELSE 0 END) AS Successful,
       ROUND(100.0 * SUM(CASE WHEN "Landing_Outcome" LIKE 'Success%' THEN 1 ELSE 0 END) / COUNT(*), 2) AS Success_Rate
FROM SPACEXTABLE
GROUP BY Year
ORDER BY Year;

 * sqlite:///my_data1.db
Done.


Year,Total_Launches,Successful,Success_Rate
2010,2,0,0.0
2012,2,0,0.0
2013,3,0,0.0
2014,6,0,0.0
2015,7,1,14.29
2016,8,5,62.5
2017,18,14,77.78
2018,20,10,50.0
2019,11,10,90.91
2020,24,21,87.5


##### Top 5 Customers by Total Payload Mass (kg)

In [17]:
%%sql
SELECT Customer, SUM(Payload_Mass__kg_) AS Total_Payload
FROM SPACEXTABLE
GROUP BY Customer
ORDER BY Total_Payload DESC
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Customer,Total_Payload
SpaceX,185220
Iridium Communications,67200
NASA (CRS),45596
"SpaceX, Planet Labs",31010
SES,23355


##### Booster versions that carried the maximum payload mass (subquery + aggregate)

In [18]:
%%sql
SELECT DISTINCT Booster_Version
FROM SPACEXTABLE
WHERE "Payload_Mass__kg_" = (
  SELECT MAX("Payload_Mass__kg_") FROM SPACEXTABLE
);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


##### 2015 records showing month name, failed drone-ship landings, booster version, launch site

In [19]:
%%sql
SELECT
  CASE substr(Date,6,2)
    WHEN '01' THEN 'January'  WHEN '02' THEN 'February' WHEN '03' THEN 'March'
    WHEN '04' THEN 'April'    WHEN '05' THEN 'May'      WHEN '06' THEN 'June'
    WHEN '07' THEN 'July'     WHEN '08' THEN 'August'   WHEN '09' THEN 'September'
    WHEN '10' THEN 'October'  WHEN '11' THEN 'November' WHEN '12' THEN 'December'
  END AS Month_Name,
  "Landing_Outcome",
  Booster_Version,
  Launch_Site
FROM SPACEXTABLE
WHERE substr(Date,1,4) = '2015'
  AND "Landing_Outcome" = 'Failure (drone ship)';

 * sqlite:///my_data1.db
Done.


Month_Name,Landing_Outcome,Booster_Version,Launch_Site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


##### Rank landing outcomes by count between 2010-06-04 and 2017-03-20 (desc)

In [20]:
%%sql
SELECT "Landing_Outcome", COUNT(*) AS cnt
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing_Outcome"
ORDER BY cnt DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,cnt
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
